# Step 6: Macroeconomic Data Integration

## Goal
Integrate macroeconomic indicators (Inflation and Interest Rates) from CBRT EVDS API with BIST-100 stock price data to analyze correlations and relationships.

## What This Notebook Does:
1. Installs required libraries (`evds` and `python-dotenv`)
2. Loads EVDS API key from `.env` file
3. Fetches Inflation (TP.FG.J0) and Interest Rates (TP.AP01.TUR) for the last 5 years
4. Resamples monthly/weekly data to daily frequency to match BIST-100 data
5. Merges macroeconomic data with BIST-100 stock prices
6. Analyzes correlation between Inflation and BIST-100 prices

In [ ]:
# Install required libraries
import subprocess
import sys

def install_package(package):
    """Install a package using pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])
        print(f"✅ {package} installed successfully")
        return True
    except subprocess.CalledProcessError:
        print(f"❌ Failed to install {package}")
        return False

# Install evds and python-dotenv
print("📦 Installing required packages...")
install_package("evds")
install_package("python-dotenv")
print("\n✅ All packages installed!")

## 1. Setup and Load API Key

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime, timedelta
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Add src to path for load_env
current_dir = Path().resolve()
if current_dir.name == "notebooks":
    project_root = current_dir.parent
else:
    project_root = current_dir

sys.path.append(str(project_root / "src"))

# Import environment loader
from load_env import get_evds_api_key
from dotenv import load_dotenv

# Load .env file
env_file = project_root / ".env"
if env_file.exists():
    load_dotenv(env_file)
    print("✅ Loaded .env file")
else:
    print("⚠️  .env file not found, trying environment variables...")

# Get API key
EVDS_API_KEY = get_evds_api_key()

if EVDS_API_KEY:
    print(f"✅ EVDS API Key loaded: {EVDS_API_KEY[:10]}...{EVDS_API_KEY[-5:]}")
else:
    print("❌ ERROR: EVDS_API_KEY not found!")
    print("   Please ensure your .env file contains: EVDS_API_KEY=your_key_here")
    raise ValueError("EVDS_API_KEY is required")

# Set up paths
data_raw_dir = project_root / "data" / "raw"
data_processed_dir = project_root / "data" / "processed"
reports_dir = project_root / "reports"

# Create directories if needed
data_processed_dir.mkdir(parents=True, exist_ok=True)
reports_dir.mkdir(parents=True, exist_ok=True)

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

print(f"\n📂 Project root: {project_root}")
print(f"📂 Raw data dir: {data_raw_dir}")
print(f"📂 Processed data dir: {data_processed_dir}")

## 2. Fetch Macroeconomic Data from EVDS API

We'll fetch:
- **TP.FG.J0**: Inflation (TÜFE - Consumer Price Index)
- **TP.AP01.TUR**: Interest Rates (Policy Rate)

For the last 5 years.

In [ ]:
import requests
import time
from datetime import datetime

# Calculate date range (last 5 years)
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)

# Format dates for EVDS API (DD-MM-YYYY)
start_date_str = start_date.strftime("%d-%m-%Y")
end_date_str = end_date.strftime("%d-%m-%Y")

print(f"📅 Date Range: {start_date_str} to {end_date_str}")
print(f"   (Last 5 years)\n")

# EVDS API base URL
EVDS_BASE_URL = "https://evds2.tcmb.gov.tr/service/evds"

# Series codes
series_codes = {
    "TP.FG.J0": "Inflation_TUFE",  # Consumer Price Index (TÜFE)
    "TP.AP01.TUR": "Interest_Rate"   # Policy Interest Rate
}

# Fetch data for each series
macro_data_list = []

for series_code, name in series_codes.items():
    print(f"📊 Fetching {name} (Series: {series_code})...")
    
    try:
        # EVDS API endpoint
        url = f"{EVDS_BASE_URL}/dataseries/{series_code}"
        params = {
            "key": EVDS_API_KEY,
            "startDate": start_date_str,
            "endDate": end_date_str,
            "type": "json",
            "formulas": "0"  # 0 = raw data
        }
        
        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()
        
        data = response.json()
        
        if 'items' in data and len(data['items']) > 0:
            # Convert to DataFrame
            df = pd.DataFrame(data['items'])
            
            # Find the date column (usually 'Tarih')
            date_col = None
            value_col = None
            
            for col in df.columns:
                if 'tarih' in col.lower() or 'date' in col.lower():
                    date_col = col
                elif col != date_col and df[col].dtype in ['float64', 'int64'] or pd.api.types.is_numeric_dtype(df[col]):
                    if value_col is None:
                        value_col = col
            
            if date_col and value_col:
                # Clean and format
                df_clean = pd.DataFrame({
                    'Date': pd.to_datetime(df[date_col], format='%d-%m-%Y', errors='coerce'),
                    name: pd.to_numeric(df[value_col], errors='coerce')
                })
                
                # Remove rows with invalid dates or NaN values
                df_clean = df_clean.dropna(subset=['Date', name])
                df_clean = df_clean.sort_values('Date').reset_index(drop=True)
                
                macro_data_list.append(df_clean)
                print(f"   ✅ Successfully fetched {len(df_clean)} records")
                print(f"   📅 Date range: {df_clean['Date'].min().date()} to {df_clean['Date'].max().date()}")
            else:
                print(f"   ⚠️  Could not parse columns. Available columns: {df.columns.tolist()}")
        else:
            print(f"   ⚠️  No data returned for {series_code}")
        
        # Rate limiting
        time.sleep(0.5)
        
    except requests.exceptions.HTTPError as e:
        print(f"   ❌ HTTP Error: {e}")
        if response.status_code == 404:
            print(f"   💡 Series code '{series_code}' might be incorrect or not available")
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")
    
    print()

# Merge all macroeconomic series
if macro_data_list:
    macro_df = macro_data_list[0]
    for df in macro_data_list[1:]:
        macro_df = macro_df.merge(df, on='Date', how='outer')
    
    macro_df = macro_df.sort_values('Date').reset_index(drop=True)
    
    print("="*60)
    print("✅ MACROECONOMIC DATA SUMMARY")
    print("="*60)
    print(f"\n📊 Shape: {macro_df.shape}")
    print(f"📅 Date range: {macro_df['Date'].min().date()} to {macro_df['Date'].max().date()}")
    print(f"📋 Columns: {macro_df.columns.tolist()}")
    print(f"\n📈 First few rows:")
    display(macro_df.head(10))
    print(f"\n📉 Last few rows:")
    display(macro_df.tail(10))
    print(f"\n📊 Statistical Summary:")
    display(macro_df.describe())
else:
    print("❌ No macroeconomic data was successfully fetched!")
    macro_df = pd.DataFrame()

In [ ]:
# Load BIST stock data
stock_file = data_raw_dir / "bist_stock_prices.csv"

if stock_file.exists():
    print(f"📊 Loading BIST stock data from: {stock_file}")
    bist_df = pd.read_csv(stock_file)
    bist_df['Date'] = pd.to_datetime(bist_df['Date'])
    bist_df = bist_df.sort_values('Date').reset_index(drop=True)
    
    # Filter for BIST-100 index if multiple tickers exist
    if 'Ticker' in bist_df.columns:
        unique_tickers = bist_df['Ticker'].unique()
        if 'XU100.IS' in unique_tickers:
            bist_df = bist_df[bist_df['Ticker'] == 'XU100.IS'].copy().reset_index(drop=True)
            print(f"✅ Loaded BIST-100 index data (XU100.IS)")
        else:
            # Use ticker with most data
            ticker_counts = bist_df['Ticker'].value_counts()
            selected_ticker = ticker_counts.index[0]
            bist_df = bist_df[bist_df['Ticker'] == selected_ticker].copy().reset_index(drop=True)
            print(f"⚠️  Using ticker: {selected_ticker} (BIST-100 not found)")
    
    # Select relevant columns (Date and Close price)
    if 'Close' in bist_df.columns:
        bist_df = bist_df[['Date', 'Close']].copy()
        bist_df = bist_df.rename(columns={'Close': 'BIST100_Close'})
    else:
        print("❌ 'Close' column not found in stock data")
        bist_df = pd.DataFrame()
    
    print(f"\n📊 BIST Data Summary:")
    print(f"   Shape: {bist_df.shape}")
    print(f"   Date range: {bist_df['Date'].min().date()} to {bist_df['Date'].max().date()}")
    print(f"\n📈 First few rows:")
    display(bist_df.head(10))
    
else:
    print(f"❌ Error: {stock_file} not found!")
    print("   Please run 01_data_collection.ipynb first to download the data.")
    bist_df = pd.DataFrame()

## 4. Resample Macroeconomic Data to Daily Frequency

Macroeconomic data from EVDS is typically monthly or weekly. We need to resample it to daily frequency to match the BIST-100 daily stock prices.

In [ ]:
if not macro_df.empty and not bist_df.empty:
    print("🔄 Resampling macroeconomic data to daily frequency...")
    
    # Set Date as index for resampling
    macro_df_indexed = macro_df.set_index('Date').copy()
    
    # Determine the date range from BIST data
    min_date = min(bist_df['Date'].min(), macro_df['Date'].min())
    max_date = max(bist_df['Date'].max(), macro_df['Date'].max())
    
    # Create daily date range
    daily_dates = pd.date_range(start=min_date, end=max_date, freq='D')
    
    # Reindex to daily frequency and forward fill (carry last known value forward)
    macro_daily = macro_df_indexed.reindex(daily_dates).ffill()
    
    # Reset index to get Date as column
    macro_daily = macro_daily.reset_index()
    macro_daily = macro_daily.rename(columns={'index': 'Date'})
    
    print(f"✅ Resampled to daily frequency")
    print(f"   Original records: {len(macro_df)}")
    print(f"   Daily records: {len(macro_daily)}")
    print(f"   Date range: {macro_daily['Date'].min().date()} to {macro_daily['Date'].max().date()}")
    
    # Show sample of resampled data
    print(f"\n📊 Sample of resampled data (showing where values change):")
    # Find rows where values actually change (not just forward-filled)
    for col in macro_daily.columns:
        if col != 'Date':
            # Mark where original data points are
            original_dates = set(macro_df['Date'].dt.date)
            macro_daily[f'{col}_is_original'] = macro_daily['Date'].dt.date.isin(original_dates)
    
    # Show a sample
    sample_indices = []
    for i in range(len(macro_daily)):
        if any(macro_daily.iloc[i][col] for col in macro_daily.columns if col.endswith('_is_original')):
            sample_indices.append(i)
        if len(sample_indices) >= 10:
            break
    
    if sample_indices:
        display(macro_daily.iloc[sample_indices][[col for col in macro_daily.columns if not col.endswith('_is_original')]].head(10))
    
    # Remove helper columns
    macro_daily = macro_daily[[col for col in macro_daily.columns if not col.endswith('_is_original')]]
    
else:
    print("❌ Cannot resample: Missing macroeconomic or BIST data")
    macro_daily = pd.DataFrame()

## 5. Merge Macroeconomic Data with BIST-100 Prices

In [ ]:
if not macro_daily.empty and not bist_df.empty:
    print("🔗 Merging macroeconomic data with BIST-100 prices...")
    
    # Ensure Date columns are exactly datetime64[ns] type
    print("\n🔍 Checking date column types...")
    print(f"   BIST Date type: {bist_df['Date'].dtype}")
    print(f"   Macro Date type: {macro_daily['Date'].dtype}")
    
    # Convert to datetime64[ns] if needed
    bist_df['Date'] = pd.to_datetime(bist_df['Date']).astype('datetime64[ns]')
    macro_daily['Date'] = pd.to_datetime(macro_daily['Date']).astype('datetime64[ns]')
    
    print(f"   After conversion - BIST Date type: {bist_df['Date'].dtype}")
    print(f"   After conversion - Macro Date type: {macro_daily['Date'].dtype}")
    
    # Merge on Date
    merged_df = bist_df.merge(macro_daily, on='Date', how='inner')
    
    # Sort by date
    merged_df = merged_df.sort_values('Date').reset_index(drop=True)
    
    print(f"\n📊 After merge (before handling missing values):")
    print(f"   Shape: {merged_df.shape}")
    print(f"   Missing values per column:")
    missing_counts = merged_df.isnull().sum()
    for col, count in missing_counts.items():
        if count > 0:
            print(f"      {col}: {count} ({count/len(merged_df)*100:.2f}%)")
    
    # Handle missing values: forward fill then backward fill
    print(f"\n🔄 Handling missing values with forward fill and backward fill...")
    merged_df_clean = merged_df.copy()
    
    # Forward fill (carry last known value forward)
    merged_df_clean = merged_df_clean.ffill()
    
    # Backward fill (fill remaining NaN at the beginning)
    merged_df_clean = merged_df_clean.bfill()
    
    # Check if there are still any NaN values
    remaining_nans = merged_df_clean.isnull().sum().sum()
    if remaining_nans > 0:
        print(f"   ⚠️  Still {remaining_nans} NaN values after ffill/bfill. Dropping rows with NaN...")
        merged_df_clean = merged_df_clean.dropna()
    else:
        print(f"   ✅ All missing values handled successfully!")
    
    # Add lagged macroeconomic features (1-month and 3-month lags)
    print(f"\n📅 Creating lagged macroeconomic features...")
    if 'Inflation_TUFE' in merged_df_clean.columns:
        # 1-month lag (approximately 30 days)
        merged_df_clean['Inflation_TUFE_Lag_1M'] = merged_df_clean['Inflation_TUFE'].shift(30)
        # 3-month lag (approximately 90 days)
        merged_df_clean['Inflation_TUFE_Lag_3M'] = merged_df_clean['Inflation_TUFE'].shift(90)
        print(f"   ✅ Added Inflation lags: 1-month, 3-month")
    
    if 'Interest_Rate' in merged_df_clean.columns:
        # 1-month lag (approximately 30 days)
        merged_df_clean['Interest_Rate_Lag_1M'] = merged_df_clean['Interest_Rate'].shift(30)
        # 3-month lag (approximately 90 days)
        merged_df_clean['Interest_Rate_Lag_3M'] = merged_df_clean['Interest_Rate'].shift(90)
        print(f"   ✅ Added Interest Rate lags: 1-month, 3-month")
    
    # Forward fill lagged features
    lag_cols = [col for col in merged_df_clean.columns if 'Lag' in col]
    if lag_cols:
        merged_df_clean[lag_cols] = merged_df_clean[lag_cols].ffill().bfill()
        print(f"   ✅ Filled missing values in lagged features")
    
    print(f"\n✅ Merge complete!")
    print(f"   Merged records (before cleaning): {len(merged_df)}")
    print(f"   Clean records (after ffill/bfill): {len(merged_df_clean)}")
    
    if not merged_df_clean.empty:
        print(f"   Date range: {merged_df_clean['Date'].min().date()} to {merged_df_clean['Date'].max().date()}")
        print(f"   Date column type: {merged_df_clean['Date'].dtype}")
        
        print(f"\n📊 Merged Dataset Summary:")
        print(f"   Shape: {merged_df_clean.shape}")
        print(f"   Columns: {merged_df_clean.columns.tolist()}")
        
        print(f"\n📈 First 5 rows (VERIFICATION):")
        display(merged_df_clean.head(5))
        
        print(f"\n📉 Last 5 rows:")
        display(merged_df_clean.tail(5))
        
        print(f"\n📊 Statistical Summary:")
        display(merged_df_clean.describe())
        
        # Save merged dataset
        output_file = data_processed_dir / "bist_macro_merged.csv"
        merged_df_clean.to_csv(output_file, index=False)
        print(f"\n💾 Saved merged dataset to: {output_file}")
    else:
        print("   ⚠️  WARNING: Merged dataframe is empty after cleaning!")
        merged_df_clean = pd.DataFrame()
    
else:
    print("❌ Cannot merge: Missing data")
    if macro_daily.empty:
        print("   - macro_daily is empty")
    if bist_df.empty:
        print("   - bist_df is empty")
    merged_df_clean = pd.DataFrame()

## 6. Correlation Analysis: Inflation vs BIST-100

Let's analyze the correlation between Inflation and BIST-100 stock prices.

In [ ]:
if not merged_df_clean.empty:
    # Calculate correlation matrix
    numeric_cols = merged_df_clean.select_dtypes(include=[np.number]).columns.tolist()
    
    if len(numeric_cols) > 1:
        correlation_matrix = merged_df_clean[numeric_cols].corr()
        
        print("="*60)
        print("📊 CORRELATION MATRIX")
        print("="*60)
        display(correlation_matrix)
        
        # Focus on Inflation vs BIST-100 correlation
        if 'Inflation_TUFE' in numeric_cols and 'BIST100_Close' in numeric_cols:
            inflation_bist_corr = correlation_matrix.loc['Inflation_TUFE', 'BIST100_Close']
            print(f"\n🔍 Key Finding:")
            print(f"   Correlation between Inflation (TÜFE) and BIST-100: {inflation_bist_corr:.4f}")
            
            if abs(inflation_bist_corr) < 0.1:
                interpretation = "Very weak correlation"
            elif abs(inflation_bist_corr) < 0.3:
                interpretation = "Weak correlation"
            elif abs(inflation_bist_corr) < 0.5:
                interpretation = "Moderate correlation"
            elif abs(inflation_bist_corr) < 0.7:
                interpretation = "Strong correlation"
            else:
                interpretation = "Very strong correlation"
            
            direction = "positive" if inflation_bist_corr > 0 else "negative"
            print(f"   Interpretation: {interpretation} ({direction})")
            
            if inflation_bist_corr > 0:
                print(f"   💡 Higher inflation tends to be associated with higher BIST-100 prices")
            else:
                print(f"   💡 Higher inflation tends to be associated with lower BIST-100 prices")
        
        # Check Interest Rate correlation if available
        if 'Interest_Rate' in numeric_cols and 'BIST100_Close' in numeric_cols:
            interest_bist_corr = correlation_matrix.loc['Interest_Rate', 'BIST100_Close']
            print(f"\n   Correlation between Interest Rate and BIST-100: {interest_bist_corr:.4f}")
            
            if interest_bist_corr > 0:
                print(f"   💡 Higher interest rates tend to be associated with higher BIST-100 prices")
            else:
                print(f"   💡 Higher interest rates tend to be associated with lower BIST-100 prices")
        
    else:
        print("⚠️  Not enough numeric columns for correlation analysis")
else:
    print("❌ No merged data available for correlation analysis")

## 7. Visualizations

In [ ]:
# Verify data exists before visualization
print("="*60)
print("🔍 PRE-VISUALIZATION DATA CHECK")
print("="*60)
print(f"merged_df_clean is empty: {merged_df_clean.empty}")
if not merged_df_clean.empty:
    print(f"Shape: {merged_df_clean.shape}")
    print(f"Columns: {merged_df_clean.columns.tolist()}")
    print(f"\n📋 First 5 rows of merged_df_clean:")
    display(merged_df_clean.head(5))
    print(f"\n📊 Data types:")
    print(merged_df_clean.dtypes)
    print(f"\n🔍 Missing values check:")
    print(merged_df_clean.isnull().sum())
    print("="*60)

if not merged_df_clean.empty:
    # Recalculate correlation matrix for visualization
    numeric_cols = merged_df_clean.select_dtypes(include=[np.number]).columns.tolist()
    
    print(f"\n📊 Numeric columns found: {numeric_cols}")
    
    if len(numeric_cols) > 1:
        correlation_matrix = merged_df_clean[numeric_cols].corr()
        
        # Create visualizations
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # 1. Time series: Inflation and BIST-100
        ax1 = axes[0, 0]
        if 'Inflation_TUFE' in merged_df_clean.columns:
            ax1_twin = ax1.twinx()
            ax1.plot(merged_df_clean['Date'], merged_df_clean['BIST100_Close'], 
                    color='blue', label='BIST-100 Close', linewidth=1.5)
            ax1_twin.plot(merged_df_clean['Date'], merged_df_clean['Inflation_TUFE'], 
                         color='red', label='Inflation (TÜFE)', linewidth=1.5, alpha=0.7)
            ax1.set_xlabel('Date', fontsize=12)
            ax1.set_ylabel('BIST-100 Close Price', fontsize=12, color='blue')
            ax1_twin.set_ylabel('Inflation (TÜFE)', fontsize=12, color='red')
            ax1.set_title('BIST-100 vs Inflation Over Time', fontsize=14, fontweight='bold')
            ax1.tick_params(axis='y', labelcolor='blue')
            ax1_twin.tick_params(axis='y', labelcolor='red')
            ax1.grid(True, alpha=0.3)
            ax1.legend(loc='upper left')
            ax1_twin.legend(loc='upper right')
        
        # 2. Scatter plot: Inflation vs BIST-100
        ax2 = axes[0, 1]
        if 'Inflation_TUFE' in merged_df_clean.columns and 'BIST100_Close' in numeric_cols:
            ax2.scatter(merged_df_clean['Inflation_TUFE'], merged_df_clean['BIST100_Close'], 
                       alpha=0.5, s=20)
            ax2.set_xlabel('Inflation (TÜFE)', fontsize=12)
            ax2.set_ylabel('BIST-100 Close Price', fontsize=12)
            ax2.set_title('Inflation vs BIST-100 Scatter Plot', fontsize=14, fontweight='bold')
            ax2.grid(True, alpha=0.3)
            
            # Add correlation coefficient
            if 'Inflation_TUFE' in numeric_cols:
                corr = correlation_matrix.loc['Inflation_TUFE', 'BIST100_Close']
                ax2.text(0.05, 0.95, f'Correlation: {corr:.3f}', 
                        transform=ax2.transAxes, fontsize=11,
                        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        
        # 3. Time series: Interest Rate and BIST-100
        ax3 = axes[1, 0]
        if 'Interest_Rate' in merged_df_clean.columns:
            ax3_twin = ax3.twinx()
            ax3.plot(merged_df_clean['Date'], merged_df_clean['BIST100_Close'], 
                    color='blue', label='BIST-100 Close', linewidth=1.5)
            ax3_twin.plot(merged_df_clean['Date'], merged_df_clean['Interest_Rate'], 
                         color='green', label='Interest Rate', linewidth=1.5, alpha=0.7)
            ax3.set_xlabel('Date', fontsize=12)
            ax3.set_ylabel('BIST-100 Close Price', fontsize=12, color='blue')
            ax3_twin.set_ylabel('Interest Rate (%)', fontsize=12, color='green')
            ax3.set_title('BIST-100 vs Interest Rate Over Time', fontsize=14, fontweight='bold')
            ax3.tick_params(axis='y', labelcolor='blue')
            ax3_twin.tick_params(axis='y', labelcolor='green')
            ax3.grid(True, alpha=0.3)
            ax3.legend(loc='upper left')
            ax3_twin.legend(loc='upper right')
        
        # 4. Correlation heatmap
        ax4 = axes[1, 1]
        sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
                   center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax4)
        ax4.set_title('Correlation Heatmap', fontsize=14, fontweight='bold')
        
        plt.tight_layout()
        
        # Save figure
        output_fig = reports_dir / 'macro_bist_correlation.png'
        plt.savefig(output_fig, dpi=300, bbox_inches='tight')
        print(f"\n💾 Saved correlation visualization to: {output_fig}")
        
        plt.show()
    else:
        print(f"\n❌ Not enough numeric columns for visualization. Found: {len(numeric_cols)}")
        print(f"   Required: At least 2 numeric columns")
        print(f"   Available columns: {merged_df_clean.columns.tolist()}")
else:
    print("\n❌ No data available for visualization")
    print("   merged_df_clean is empty. Please check the merge step above.")

## 8. Summary

This notebook successfully:
- ✅ Fetched Inflation (TP.FG.J0) and Interest Rates (TP.AP01.TUR) from CBRT EVDS API
- ✅ Resampled monthly/weekly macroeconomic data to daily frequency
- ✅ Merged macroeconomic indicators with BIST-100 stock prices
- ✅ Created lagged features (1-month and 3-month lags) for Inflation and Interest Rates
- ✅ Analyzed correlations between Inflation and BIST-100 prices

### Key Features Created:
- **Current Features**: Inflation_TUFE, Interest_Rate
- **Lagged Features**: 
  - Inflation_TUFE_Lag_1M (1-month lag)
  - Inflation_TUFE_Lag_3M (3-month lag)
  - Interest_Rate_Lag_1M (1-month lag)
  - Interest_Rate_Lag_3M (3-month lag)

### Key Insights:
- The correlation analysis reveals the relationship between macroeconomic factors and stock market performance
- Lagged features capture delayed effects (e.g., how inflation from last month affects current stock prices)
- This merged dataset can be used to enhance machine learning models by including macroeconomic features

### Next Steps:
- Use the merged dataset (`bist_macro_merged.csv`) in model training to include macroeconomic features
- The lagged features help capture temporal relationships between macro indicators and stock prices
- Add more macroeconomic indicators (exchange rates, GDP, etc.) for comprehensive analysis